# 範例 9：LM Studio 串流輸出

即時顯示 AI 的回應，像 ChatGPT 一樣一個字一個字出現！

## 學習目標
- 使用 OpenAI SDK 實現串流輸出
- 了解串流 API 的使用方式
- 比較串流與非串流的差異

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件並設定

In [ ]:
from openai import OpenAI

# 建立客戶端
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

## Step 2: 定義串流對話函數

In [ ]:
def stream_chat_lmstudio(message):
    """
    使用串流方式獲得 LM Studio 回應
    """

    # 發送串流請求
    stream = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": message}],
        stream=True  # 啟用串流模式
    )

    print("AI：", end="", flush=True)

    # 逐步接收並顯示回應
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            full_response += content

    print()  # 最後換行
    return full_response

## Step 3: 測試串流輸出

In [ ]:
question = "請用三句話介紹台灣。"
print(f"問題：{question}")
print("-" * 50)
stream_chat_lmstudio(question)

## Step 4: 串流 vs 非串流比較

In [ ]:
import time

def normal_chat(message):
    """非串流方式"""
    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": message}],
        stream=False
    )
    return response.choices[0].message.content

In [ ]:
test_prompt = "請解釋什麼是機器學習，用三個重點說明。"

print("=== 非串流模式 ===")
print("（需要等待完整回應...）\n")
start = time.time()
result = normal_chat(test_prompt)
end = time.time()
print(f"AI：{result}")
print(f"\n等待時間：{end - start:.2f} 秒")

In [ ]:
print("\n=== 串流模式 ===")
print("（文字即時顯示）\n")
start = time.time()
stream_chat_lmstudio(test_prompt)
end = time.time()
print(f"\n總時間：{end - start:.2f} 秒")

## Step 5: 進階串流 - 收集完整回應

In [ ]:
def stream_chat_with_callback(message, on_chunk=None):
    """
    串流對話，支援自訂處理函數
    
    參數：
        message: 使用者訊息
        on_chunk: 每收到一個片段時的處理函數
    
    回傳：
        完整的回應文字
    """
    stream = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": message}],
        stream=True
    )
    
    full_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            full_response += content
            
            # 如果有提供處理函數，就呼叫它
            if on_chunk:
                on_chunk(content)
    
    return full_response

In [ ]:
# 自訂處理函數：計算字數
char_count = 0

def count_chars(chunk):
    global char_count
    char_count += len(chunk)
    print(chunk, end="", flush=True)

print("即時輸出並計算字數：\n")
response = stream_chat_with_callback("什麼是 API？簡單解釋。", on_chunk=count_chars)
print(f"\n\n總共收到 {char_count} 個字元")

## 串流 API 解析

### 關鍵設定
```python
stream = client.chat.completions.create(
    model="gpt-oss-120b",
    messages=[...],
    stream=True  # 這個參數啟用串流
)
```

### 處理串流回應
```python
for chunk in stream:
    # chunk.choices[0].delta.content 包含這次收到的文字片段
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")
```

### 注意事項
- `delta` 物件包含這次收到的增量內容
- 需要檢查 `delta.content` 是否存在（可能是 None）
- 使用 `flush=True` 確保即時輸出

## 練習

In [ ]:
# 試著用串流方式問一個需要長回答的問題
my_question = "請詳細解釋網路是如何運作的？"

print(f"問題：{my_question}")
print("-" * 50)
stream_chat_lmstudio(my_question)

## 重點回顧

1. **啟用串流**：設定 `stream=True`
2. **處理片段**：使用 `for chunk in stream` 迭代
3. **取得內容**：`chunk.choices[0].delta.content`
4. **即時輸出**：使用 `print(..., end="", flush=True)`

## 下一步

在下一個範例中，我們將學習如何查看 LM Studio 中可用的模型！